# Merging everything

This notebook is for merging the boiler data (DOB NOW and CATS) with the buildings (Pluto-PAD) and demographic (ACS) data.

We've got a problem: When we merge the ACS with the Pluto-PAD data (on the combined borough code and census tract), the resulting dataframe has a lot of null values, especially in some of the demographic columns (like 1 million out of a total dataframe size of 1.3 million). This shouldn't be happening. The Pluto-PAD data only has 790 rows missing census tract values. And the ACS data is mostly not missing any data and has no null values for columns like "over_65". 

So after the merge, the resulting dataframe should have, at most, 790 rows with null values in the demographic data columns.

# To do

Figure out why the merge isn't able to match the borough code-census tract IDs in the Pluto/PAD and ACS dataframes.

In [1]:
import pandas as pd

In [2]:
# Used to figure out which columns have nulls
def nullchecker(df):
    for i in range(len(df.columns)):
        if df.iloc[:,i].isnull().sum() > 0:
            print(df.columns[i] + ':')
            print("Number of nulls: {}".format(df.iloc[:,i].isnull().sum()))
            print("")

## Pluto and PAD data

In [3]:
pad_pluto = pd.read_csv('data/Buildings_w_Pluto.csv')
pad_pluto.head()

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23,58,59,70) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,boro_x,block_x,lot_x,bin,lhnd,lhns,lcontpar,lsos,hhnd,...,commfar,facilfar,borocode,bbl,condono,tract2010,xcoord,ycoord,taxmap,boro_y
0,0,1,1,10,1089249,,,,L,,...,0.0,1.0,1.0,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1
1,1,1,1,10,1000000,,,,R,,...,0.0,1.0,1.0,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1
2,2,1,1,10,1000000,,,,R,,...,0.0,1.0,1.0,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1
3,3,1,1,10,1000000,,,,R,,...,0.0,1.0,1.0,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1
4,4,1,1,10,1000000,,,,R,,...,0.0,1.0,1.0,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1


## ACS data

In [4]:
acs = pd.read_csv('data/ACS_nyc.csv')
acs.head()

,over_65,total_population,white_population,median_household_income,state,county,tract,boro,over_65_pct,over_white_pct
0,0,7411,1464,NaN,36,5,100,2,0.000000,0.197544
1,786,5058,2665,62773.0,36,5,200,2,0.155397,0.526888
2,707,5944,2616,82697.0,36,5,400,2,0.118943,0.440108
3,748,6115,2685,33118.0,36,5,1600,2,0.122322,0.439084
4,89,2817,519,40117.0,36,5,1900,2,0.031594,0.184239


In [6]:
acs.shape

(2167, 10)

In [7]:
acs.tract = acs.tract.astype(int)

In [8]:
#format census tract ID to 6 characters
def replaceSuffix(x):
    val = len(x)
    # now if still not length of 4, add zeroes to beginning (300 -> 0300)
    for i in range(4-val):
        x='0'+x
    return x

In [9]:
# first drop na values
pad_pluto = pad_pluto.dropna(axis=0, subset=['ct2010'])
# next convert to string (dont want to get rid of suffix)
pad_pluto['ct20102'] = pad_pluto.ct2010.apply(lambda x: x*100)
# next get rid of decimals
pad_pluto['ct20102'] = pad_pluto.ct20102.astype(int)
pad_pluto['ct20102'] = pad_pluto.ct20102.astype(str)
# apply function
pad_pluto['ct20102'] = pad_pluto.ct20102.apply(replaceSuffix)

In [10]:
pad_pluto[['ct2010','boro_y', 'ct20102']].head()

,ct2010,boro_y,ct20102
0,5.0,1,0500
1,5.0,1,0500
2,5.0,1,0500
3,5.0,1,0500
4,5.0,1,0500


In [11]:
# these are the ones that messed us up before
pad_pluto[pad_pluto.ct20102.apply(lambda x: x.endswith('01'))].head()

,Unnamed: 0,boro_x,block_x,lot_x,bin,lhnd,lhns,lcontpar,lsos,hhnd,...,facilfar,borocode,bbl,condono,tract2010,xcoord,ycoord,taxmap,boro_y,ct20102
1200,1200,1,41,15,1001008,20,000020000AA,,L,24,...,4.0,1.0,1.004110e+09,0.0,3001.0,987407.0,201876.0,10204,1,3001
1202,1202,1,41,15,1001008,60,000060000AA,,L,64,...,4.0,1.0,1.004110e+09,0.0,3001.0,987407.0,201876.0,10204,1,3001
1204,1204,1,411,5,1005344,154,000154000AA,,R,154,...,4.0,1.0,1.004110e+09,0.0,3001.0,987407.0,201876.0,10204,1,3001
1206,1206,1,411,5,1088378,154 REAR,000154000AC,,R,154 REAR,...,4.0,1.0,1.004110e+09,0.0,3001.0,987407.0,201876.0,10204,1,3001
1207,1207,1,41,22,1001012,48,000048000AA,,L,48,...,4.0,1.0,1.004120e+09,0.0,3001.0,987520.0,202154.0,10204,1,3001


In [12]:
# Convert to integer, then string
pad_pluto['borocode'] = pad_pluto['borocode'].astype(int)
pad_pluto['borocode'] = pad_pluto['borocode'].astype(str)

In [13]:
# Combine the borough code with the census tract to create a unique ID for merging
pad_pluto['ct_bc_comb'] = pad_pluto['borocode'] + '0' + pad_pluto['ct20102']
pad_pluto.head()

,Unnamed: 0,boro_x,block_x,lot_x,bin,lhnd,lhns,lcontpar,lsos,hhnd,...,borocode,bbl,condono,tract2010,xcoord,ycoord,taxmap,boro_y,ct20102,ct_bc_comb
0,0,1,1,10,1089249,,,,L,,...,1,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1,0500,100500
1,1,1,1,10,1000000,,,,R,,...,1,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1,0500,100500
2,2,1,1,10,1000000,,,,R,,...,1,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1,0500,100500
3,3,1,1,10,1000000,,,,R,,...,1,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1,0500,100500
4,4,1,1,10,1000000,,,,R,,...,1,1.000010e+09,0.0,5.0,979071.0,190225.0,10101,1,0500,100500


### Now moving onto ACS

In [14]:
acs['tract'] = acs.tract.astype(str)
acs[~acs.tract.apply(lambda x: x.endswith('00'))].head()

,over_65,total_population,white_population,median_household_income,state,county,tract,boro,over_65_pct,over_white_pct
9,247,2958,216,17383.0,36,5,2701,2,0.083502,0.073022
10,303,4538,495,13216.0,36,5,2702,2,0.066770,0.109079
18,228,1428,386,49706.0,36,5,4001,2,0.159664,0.270308
25,469,5101,510,24683.0,36,5,5001,2,0.091943,0.099980
26,424,5583,1168,27321.0,36,5,5002,2,0.075945,0.209207


In [20]:
acs['tract2'] = acs.tract.apply(replaceSuffix)
acs['boro'] = acs.boro.astype(str)
# Combine the borough code with the census tract to create a unique ID for merging
acs['ct_bc_comb'] = acs['boro'] + '0' +  acs['tract2']
acs[['boro','tract2','ct_bc_comb']].head()

,boro,tract2,ct_bc_comb
0,2,0100,200100
1,2,0200,200200
2,2,0400,200400
3,2,1600,201600
4,2,1900,201900


In [61]:
acs.to_csv('data/ACS_for_joining.csv')

In [21]:
acs.columns

Index(['over_65', 'total_population', 'white_population',
       'median_household_income', 'state', 'county', 'tract', 'boro',
       'over_65_pct', 'over_white_pct', 'tract2', 'ct_bc_comb'],
      dtype='object')

In [28]:
pad_pluto.shape

(1337805, 74)

In [22]:
new_merge = pd.merge(pad_pluto, acs, how='left', on='ct_bc_comb')
new_merge.head()

,Unnamed: 0,boro_x,block_x,lot_x,bin,lhnd,lhns,lcontpar,lsos,hhnd,...,total_population,white_population,median_household_income,state,county,tract,boro,over_65_pct,over_white_pct,tract2
0,0,1,1,10,1089249,,,,L,,...,0.0,0.0,NaN,36.0,61.0,500,1,NaN,NaN,0500
1,1,1,1,10,1000000,,,,R,,...,0.0,0.0,NaN,36.0,61.0,500,1,NaN,NaN,0500
2,2,1,1,10,1000000,,,,R,,...,0.0,0.0,NaN,36.0,61.0,500,1,NaN,NaN,0500
3,3,1,1,10,1000000,,,,R,,...,0.0,0.0,NaN,36.0,61.0,500,1,NaN,NaN,0500
4,4,1,1,10,1000000,,,,R,,...,0.0,0.0,NaN,36.0,61.0,500,1,NaN,NaN,0500


In [24]:
total = new_merge.shape[0]
num_failed = total - new_merge[new_merge.county > 0].shape[0], 
print('{} of {} Failed.\n{}'.format(num_failed[0], total, (num_failed[0]/total)*100))

205 of 1337805 Failed.
0.01532360844816696


### Only 0.02% failed

## Comparing the combined borough and tracts in Pluto/PAD and ACS

In [25]:
padlist = pad_pluto.ct_bc_comb.unique()
acslist = acs.ct_bc_comb.unique()
missing = [x for x in padlist if x not in acslist]
both = [x for x in acslist if x in padlist]

In [26]:
len(missing), len(both)

(8, 2160)

In [27]:
new_merge.to_csv('data/pluto_pad_acs.csv')

#### 8 missing from merge, I'd say that's ok

## Merge with DOB NOW/CATS data

### Using Pablo's updated historical data

In [67]:
updated_dbn = pd.read_csv('data/all_boiler_year_data.csv')
updated_dbn.head()

,bin_number,boiler_id,defects_exist,inspection_year,first_inspection,boiler_age,years_since_last_defects,boiler_make,boiler_model,boiler_pressure_type
0,1080776,10000000027N0001,0,1993,1993,0,NaN,SLANTFIN,GG-399,Low Pressure
1,1080776,10000000027N0001,0,1994,1993,1,NaN,SLANTFIN,GG-399,Low Pressure
2,1080776,10000000027N0001,0,1995,1993,2,NaN,SLANTFIN,GG-399,Low Pressure
3,1080776,10000000027N0001,0,1996,1993,3,NaN,SLANTFIN,GG-399,Low Pressure
4,1080776,10000000027N0001,0,1997,1993,4,NaN,SLANTFIN,GG-399,Low Pressure


In [68]:
updated_dbn.shape

(1726377, 10)

In [69]:
# Merge the DOB historical data with the merged Pluto/PAD-ACS data
updated_dbn = updated_dbn.merge(new_merge, how='inner', left_on='bin_number', right_on='bin')
updated_dbn.head()

,bin_number,boiler_id,defects_exist,inspection_year,first_inspection,boiler_age,years_since_last_defects,boiler_make,boiler_model,boiler_pressure_type,...,total_population,white_population,median_household_income,state,county,tract,boro,over_65_pct,over_white_pct,tract2
0,1080776,10000000027N0001,0,1993,1993,0,NaN,SLANTFIN,GG-399,Low Pressure,...,3803.0,2978.0,135588.0,36.0,61.0,7400,1,0.14173,0.783066,7400
1,1080776,10000000027N0001,0,1993,1993,0,NaN,SLANTFIN,GG-399,Low Pressure,...,3803.0,2978.0,135588.0,36.0,61.0,7400,1,0.14173,0.783066,7400
2,1080776,10000000027N0001,0,1994,1993,1,NaN,SLANTFIN,GG-399,Low Pressure,...,3803.0,2978.0,135588.0,36.0,61.0,7400,1,0.14173,0.783066,7400
3,1080776,10000000027N0001,0,1994,1993,1,NaN,SLANTFIN,GG-399,Low Pressure,...,3803.0,2978.0,135588.0,36.0,61.0,7400,1,0.14173,0.783066,7400
4,1080776,10000000027N0001,0,1995,1993,2,NaN,SLANTFIN,GG-399,Low Pressure,...,3803.0,2978.0,135588.0,36.0,61.0,7400,1,0.14173,0.783066,7400


In [70]:
updated_dbn.shape

(2569775, 95)

In [72]:
updated_dbn.drop_duplicates(inplace=True, subset=['bin_number', 'boiler_id', 'defects_exist', 'inspection_year',
       'first_inspection', 'boiler_age', 'years_since_last_defects',
       'boiler_make', 'boiler_model', 'boiler_pressure_type'])

updated_dbn.shape
# print(dbncats_ppacs.shape)

(1632518, 95)

In [73]:
updated_dbn.to_csv('data/all_boilder_year_data_w_ACS.csv')

### Using old set of DBN Now Data (2017 and later)

In [31]:
dbn_cats = pd.read_csv('data/dnb_cats_merge.csv')
dbn_cats.drop(['Unnamed: 0'], axis=1, inplace=True)
dbn_cats.head()

,Tracking Number,Boiler ID,Report Type,ApplicantFirst Name,Applicant Last Name,Applicant License Type,Applicant License Number,Owner First Name,Owner Last Name,Boiler Make,...,Model,BurnerMake,BurnerModel,PrimaryFuel,SecondaryFuel,Quantity,IssueDate,status,PremiseName,new_bins
0,2017-10000008199Y0001-24915,10000008199Y0001,Initial,DREW,THOMAS,BB,5468.0,BARBARA,KAMMERER,ROCKMILLS STE,...,MP-100,IND. COMB.,MMG-42-P,NATURALGAS,NO4FUEL,1.0,06/20/2018 02:38:45 PM,CURRENT,REX MANAGEMENT CORP.,1059945.0
1,2017-10000017650Y0001-24577,10000017650Y0001,Initial,BENJAMIN,WAGNER,BB,5469.0,LINDA,BALIDEMAJ,ROCKMILLS,...,ROCKMILLS MP 4-122,ICI AM6CM,ICI AM6CM,NO4FUEL,NONE,1.0,01/01/1900 12:01:00 AM,EXPIRED,NaN,1017834.0
2,2017-10000017652Y0001-24652,10000017652Y0001,Initial,JOHN,KNIEF,O,5317.0,CHRISTINA,RIVERA,H.B. SMITH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-10000018251Y0001-24562,10000018251Y0001,Initial,BENJAMIN,WAGNER,BB,5469.0,NATE,KLEIN,PEERLESS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-10000018279Y0001-24631,10000018279Y0001,Initial,EDWIN,VAZQUEZ,P,1526.0,LEONARD,EIS,CROTTY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# This dataframe is bigger than the one I had by the end of my notebook "dobnow_cats_merge." Anyone know what changed?
dbn_cats.shape

(86935, 43)

In [50]:
# Merge the DOB NOW/CATS data with the merged Pluto/PAD-ACS data
dbncats_ppacs = dbn_cats.merge(new_merge, how='inner', left_on='Bin Number', right_on='bin')
dbncats_ppacs.head()

,Tracking Number,Boiler ID,Report Type,ApplicantFirst Name,Applicant Last Name,Applicant License Type,Applicant License Number,Owner First Name,Owner Last Name,Boiler Make,...,total_population,white_population,median_household_income,state,county,tract,boro,over_65_pct,over_white_pct,tract2
0,2017-10000008199Y0001-24915,10000008199Y0001,Initial,DREW,THOMAS,BB,5468.0,BARBARA,KAMMERER,ROCKMILLS STE,...,3405.0,1027.0,26007.0,36.0,61.0,22302,1,0.158884,0.301615,22302
1,2017-10000008199Y0001-24915,10000008199Y0001,Initial,DREW,THOMAS,BB,5468.0,BARBARA,KAMMERER,ROCKMILLS STE,...,3405.0,1027.0,26007.0,36.0,61.0,22302,1,0.158884,0.301615,22302
2,2018-10000008199Y0001-73203,10000008199Y0001,Initial,ROBERT,KAZALONIS,BB,5567.0,BARBARA,KAMMERER,ROCKMILLS STE,...,3405.0,1027.0,26007.0,36.0,61.0,22302,1,0.158884,0.301615,22302
3,2018-10000008199Y0001-73203,10000008199Y0001,Initial,ROBERT,KAZALONIS,BB,5567.0,BARBARA,KAMMERER,ROCKMILLS STE,...,3405.0,1027.0,26007.0,36.0,61.0,22302,1,0.158884,0.301615,22302
4,2017-10000017650Y0001-24577,10000017650Y0001,Initial,BENJAMIN,WAGNER,BB,5469.0,LINDA,BALIDEMAJ,ROCKMILLS,...,5009.0,4150.0,137986.0,36.0,61.0,5000,1,0.135955,0.828509,5000


In [56]:
dbncats_ppacs.drop_duplicates(inplace=True, subset=['Tracking Number', 'Boiler ID', 'Inspection Date',
                                                   'Bin Number','tract','boro'])
print(dbncats_ppacs.shape)
dbncats_ppacs.head()

(90092, 128)


,Tracking Number,Boiler ID,Report Type,ApplicantFirst Name,Applicant Last Name,Applicant License Type,Applicant License Number,Owner First Name,Owner Last Name,Boiler Make,...,total_population,white_population,median_household_income,state,county,tract,boro,over_65_pct,over_white_pct,tract2
0,2017-10000008199Y0001-24915,10000008199Y0001,Initial,DREW,THOMAS,BB,5468.0,BARBARA,KAMMERER,ROCKMILLS STE,...,3405.0,1027.0,26007.0,36.0,61.0,22302,1,0.158884,0.301615,22302
2,2018-10000008199Y0001-73203,10000008199Y0001,Initial,ROBERT,KAZALONIS,BB,5567.0,BARBARA,KAMMERER,ROCKMILLS STE,...,3405.0,1027.0,26007.0,36.0,61.0,22302,1,0.158884,0.301615,22302
4,2017-10000017650Y0001-24577,10000017650Y0001,Initial,BENJAMIN,WAGNER,BB,5469.0,LINDA,BALIDEMAJ,ROCKMILLS,...,5009.0,4150.0,137986.0,36.0,61.0,5000,1,0.135955,0.828509,5000
5,2018-10000017650Y0001-84273,10000017650Y0001,Initial,BRIAN,SEIBERT,BB,5492.0,LINDA,BALIDEMAJ,ROCKMILLS,...,5009.0,4150.0,137986.0,36.0,61.0,5000,1,0.135955,0.828509,5000
6,2017-10000017652Y0001-24652,10000017652Y0001,Initial,JOHN,KNIEF,O,5317.0,CHRISTINA,RIVERA,H.B. SMITH,...,3642.0,2869.0,159821.0,36.0,61.0,5200,1,0.112026,0.787754,5200


In [58]:
# whats the size of the outputs
dbn_cats.shape[0], dbncats_ppacs.shape[0]

(86935, 90092)

In [59]:
# how many dbnow records have census data?
dbncats_ppacs[dbncats_ppacs.county > 0].shape

(90059, 128)

In [60]:
# export dataframe
dbncats_ppacs.to_csv('data/dbncats_ppacs.csv')